In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk

In [2]:
col = ['target','ids','date','flag','user','tweets']

In [3]:
twitter_data = pd.read_csv(f'/home/sonis/datasets/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
twitter_data.columns  = col
twitter_data.head()

,target,ids,date,flag,user,tweets
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
data= twitter_data[['target','tweets']]
data.head()

,target,tweets
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [6]:
data.target.replace(4,1,inplace=True)

/tmp/ipykernel_48373/3469991616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.target.replace(4,1,inplace=True)


In [7]:
data.target.value_counts()

1    800000
0    799999
Name: target, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_train,x_test, y_train,y_test= train_test_split(data.tweets,data.target,test_size = .3, random_state=5000)

In [11]:
x_train.head()

1483719                                 @hasselgren Me like 
796045     The audition comes here in exactly a week but ...
1433966             L train Canarsie Rockaway Prkway Bound. 
1084800    @ooohalby 2 appetizers (not on the menu). 2 sa...
643147     Tweet4Skype could be a nice impementation, but...
Name: tweets, dtype: object

In [12]:
y_train.head()

1483719    1
796045     0
1433966    1
1084800    1
643147     0
Name: target, dtype: int64

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [15]:
stopwords_english = stopwords.words('english') 

print('Stop words\n')
print(stopwords_english)

print('\nPunctuation\n')
print(string.punctuation)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [16]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False,        strip_handles=True,reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [17]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs

In [18]:
# from utils import process_tweet # Import the process_tweet function

# choose the same tweet
# tweet = all_positive_tweets[2277]

print()
print('\033[92m')
print(x_train[9])
print('\033[94m')

# call the imported function
tweets_stem = process_tweet(x_train[9]); # Preprocess a given tweet

print('preprocessed tweet:')
print(tweets_stem) # Print the result



spring break in plain city... it's snowing 

preprocessed tweet:
['spring', 'break', 'plain', 'citi', '...', 'snow']


In [19]:
# create frequency dictionary
freqs = build_freqs(x_train, y_train)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')

type(freqs) = <class 'dict'>
len(freqs) = 271884


In [20]:
freqs

{('like', 1): 27621,
 ('audit', 0): 197,
 ('come', 0): 13851,
 ('exactli', 0): 614,
 ('week', 0): 11046,
 ('allow', 0): 933,
 ('go', 0): 54987,
 ('venu', 0): 122,
 ('heeellppp', 0): 1,
 ('l', 1): 1056,
 ('train', 1): 2264,
 ('canarsi', 1): 3,
 ('rockaway', 1): 7,
 ('prkway', 1): 1,
 ('bound', 1): 161,
 ('2', 1): 11724,
 ('appet', 1): 16,
 ('menu', 1): 98,
 ('salad', 1): 366,
 ('filet', 1): 13,
 ('mignon', 1): 7,
 ('bake', 1): 523,
 ('potato', 1): 262,
 ('creme', 1): 40,
 ('brule', 1): 10,
 ('4', 1): 5374,
 ('cocktail', 1): 135,
 ('tweet', 0): 5973,
 ('4skype', 0): 1,
 ('could', 0): 9324,
 ('nice', 0): 4553,
 ('impement', 0): 1,
 ('work', 0): 40964,
 ('congrat', 1): 2279,
 ('bella', 1): 136,
 ('15', 1): 738,
 ('yr', 1): 660,
 ('could', 1): 5562,
 ('beat', 1): 1149,
 ('boy', 1): 3106,
 ('tour', 1): 1260,
 ('de', 1): 629,
 ('franc', 1): 255,
 ('enrout', 1): 8,
 ('newburi', 1): 9,
 ('spa', 1): 137,
 ('afternoon', 1): 1705,
 ('pamper', 1): 65,
 ('spoil', 1): 207,
 ('alon', 0): 2772,
 ('girl

In [22]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [25]:
# Check your function
# test on training data
tmp1 = extract_features(x_train[1], freqs)
print(tmp1)

[[1.0000e+00 7.2704e+04 8.5956e+04]]


In [26]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('i love it', freqs)
print(tmp2)

[[1.0000e+00 4.1134e+04 1.4776e+04]]


In [28]:
len(x_train),len(y_train)

(1119999, 1119999)

In [41]:
x_train,x_train.tolist()[2]

(1483719                                 @hasselgren Me like 
 796045     The audition comes here in exactly a week but ...
 1433966             L train Canarsie Rockaway Prkway Bound. 
 1084800    @ooohalby 2 appetizers (not on the menu). 2 sa...
 643147     Tweet4Skype could be a nice impementation, but...
                                  ...                        
 1113474                                  @ginavon thank you 
 489373     @FaketragedyCom Yeah. Sigh.  But, off tomorrow...
 1366625    @fujimama your squirrel is kawaii des! Bug is ...
 1213890    @ddlovato andy samberg is gonna sing mother lo...
 188827     I'M GRADUATING! I'M GRADUATING! IT'S ABOUT F'I...
 Name: tweets, Length: 1119999, dtype: object,
 'L train Canarsie Rockaway Prkway Bound. ')

In [43]:
m = len(x_train)
x_train = x_train.tolist()
X = np.zeros((m,3))
for i in range(m):
#     p_tweet = process_tweet(tweets[i])
    X[i,:] = extract_features(x_train[i],freqs)

In [44]:
print('hello')

hello


In [45]:
X[:5]

array([[1.0000e+00, 2.7621e+04, 2.9991e+04],
       [1.0000e+00, 6.3883e+04, 8.1751e+04],
       [1.0000e+00, 3.4920e+03, 2.8860e+03],
       [1.0000e+00, 3.0292e+04, 3.8546e+04],
       [1.0000e+00, 4.6316e+04, 6.0816e+04]])

In [46]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [60]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of h
        h = sigmoid(z)
        
        # calculate the cost function
        # note that we can use also np.array.transpose() instead of np.array.T
        # np.array.T just makes code a little more readable :)
        J = -1./m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T,np.log(1-h)))                                                    

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.T,(h-y))
        
    
#     J = float(J)
    return J, theta

In [68]:
J, theta = gradientDescent(X, np.array(y_train.tolist()), np.zeros((3, 1)), 1e-3, 2)


MemoryError: Unable to allocate 9.13 TiB for an array with shape (1119999, 1119999) and data type float64

In [67]:
theta.shape

(3, 500)